<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

- Run the ETL process
- Extract bank and market cap data from the JSON file `bank_market_cap.json`
- Transform the market cap currency using the exchange rate data
- Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [ ]:
#!mamba install pandas==1.3.3 -y
#!mamba install requests==2.26.0 -y

## Imports

Import any additional libraries you may need here.


In [87]:
import glob
import pandas as pd
import json
from datetime import datetime
import urllib.request

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section  


## Extract


### JSON Extract Function

This function will extract JSON files.


In [68]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

In [93]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [69]:
columns=['Name','Market Cap (US$ Billion)']

In [104]:
# Download JSON files
json_urls = [
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json",
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json"
]

for url in json_urls:
    filename = url.split("/")[-1]
    urllib.request.urlretrieve(url, filename)

# Download CSV file
csv_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv"
urllib.request.urlretrieve(csv_url, "exchange_rates.csv")

def extract():
    extracted_data = pd.DataFrame(columns=['Name', 'Market Cap (US$ Billion)'])
    for jsonfile in glob.glob("bank_market_cap_*.json"):
        with open(jsonfile) as f:
            data = json.load(f)
            df = pd.DataFrame.from_dict(data)
            extracted_data = pd.concat([extracted_data, df], ignore_index=True)
    
    return extracted_data

In [89]:
extract()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
...,...,...
100,Ping An Bank,37.993
101,Standard Chartered,37.319
102,United Overseas Bank,35.128
103,QNB Group,33.560


<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [105]:
def exchange_rates(currency):
    exchange_rate = extract_from_csv('exchange_rates.csv')
    exchange_rate.rename(columns = {'Unnamed: 0':'Currency'}, inplace = True)
    exchange_rate = exchange_rate.set_index('Currency')
    return exchange_rate.loc[[currency]]
    
exchange_rates('GBP')

,Rates
Currency,
GBP,0.732398


## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1. Changes the `Market Cap (US$ Billion)` column from USD to GBP
2. Rounds the Market Cap (US$ Billion)` column to 3 decimal places
3. Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [106]:
def transform(bank_market_cap, exchange_rates):
    rate = exchange_rates['Rates'].iloc[0]  # Extract the rate value from the DataFrame
    bank_market_cap['Market Cap (GBP$ Billion)'] = bank_market_cap['Market Cap (US$ Billion)'].astype(float).mul(float(rate))
    bank_market_cap = bank_market_cap.drop('Market Cap (US$ Billion)', axis=1)
    return bank_market_cap
    # Write your code here

transform(extract(),  exchange_rates('GBP'))

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319444
1,Industrial and Commercial Bank of China,252.834188
2,Bank of America,238.271911
3,Wells Fargo,225.588235
4,China Construction Bank,188.518621
...,...,...
100,Ping An Bank,27.826013
101,Standard Chartered,27.332377
102,United Overseas Bank,25.727692
103,QNB Group,24.579291


## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [107]:
targetfile="bank_matket_cap_gbp.csv"
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile, index=False)

## Logging Function


Write the logging function <code>log</code> to log your data:


In [108]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [109]:
# Write your code here
log("ETL Job Started")
log("Extract Phase Started")

### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [110]:
# Call the function here
extract_data=extract()
# Print the rows here
extract_data.head(5)

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


Log the data as <code>"Extract phase Ended"</code>


In [111]:
# Write your code here
log("Extract Phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [112]:
# Write your code here
log("Transform Phase Started")

<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [116]:
# Call the function here
transformed_data=transform(extract(),  exchange_rates('GBP'))

# Print the first 5 rows here
transformed_data.head(5)

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319444
1,Industrial and Commercial Bank of China,252.834188
2,Bank of America,238.271911
3,Wells Fargo,225.588235
4,China Construction Bank,188.518621


Log your data <code>"Transform phase Ended"</code>


In [117]:
# Write your code here
log("Transform Phase Ended")

### Load


Log the following `"Load phase Started"`.


In [56]:
# Write your code here
log("load Phase Started")

Call the load function


In [118]:
# Write your code here
load(targetfile,transformed_data)

Log the following `"Load phase Ended"`.


In [119]:
# Write your code here
log("load Phase Ended")

## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


 Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
